# Tutorial

In [5]:
using DiffEqFlux, DifferentialEquations, Optimization, OptimizationOptimJL, Random, Plots, CSV, Lux, DataFrames
using DataDrivenDiffEq, ModelingToolkit, LinearAlgebra, DiffEqSensitivity, Zygote, Optim, CSV, Lux, Pkg
gr()
Pkg.status()

      Status `~/covid19/discovering_missing_terms/Project.toml`
  [336ed68f] CSV v0.10.7
  [2445eb08] DataDrivenDiffEq v0.8.6
  [a93c6f00] DataFrames v1.4.3
  [aae7a2af] DiffEqFlux v1.52.0
  [41bf760c] DiffEqSensitivity v6.79.0
  [0c46a032] DifferentialEquations v7.6.0
  [5789e2e9] FileIO v1.16.0
  [587475ba] Flux v0.13.7
  [7073ff75] IJulia v1.23.3
  [033835bb] JLD2 v0.4.28
  [b2108857] Lux v0.4.34
  [961ee093] ModelingToolkit v8.33.0
  [429524aa] Optim v1.7.3
  [7f7a1694] Optimization v3.9.2
  [36348300] OptimizationOptimJL v0.1.4
  [1dea7af3] OrdinaryDiffEq v6.31.2
  [91a5bcdd] Plots v1.36.1
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra


### ODE data for simmulation

In [7]:
rng = Random.default_rng()
u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[1], tspan[2], length = datasize)

function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u.^3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))

2×30 Matrix{Float32}:
 2.0  1.9465    1.74178  1.23837  0.577127  …  1.40688   1.37023   1.29214
 0.0  0.798832  1.46473  1.80877  1.86465      0.451377  0.728699  0.972102

### This is an Neural ODE

In [ ]:
dudt2 = Lux.Chain(x -> x.^3,
                  Lux.Dense(2, 50, tanh),
                  Lux.Dense(50, 2))
p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

In [ ]:
function predict_neuralode(p)
  Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
    plt = scatter(tsteps, ode_data[1,:], label = "data")
    scatter!(plt, tsteps, pred[1,:], label = "prediction")
    display(plot(plt))
  end
  return false
end

pinit = Lux.ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...; doplot=true)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(optprob,
                                       ADAM(0.05),
                                       callback = callback,
                                       maxiters = 300)

optprob2 = remake(optprob,u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2,
                                        Optim.BFGS(initial_stepnorm=0.01),
                                        callback=callback,
                                        allow_f_increases = false)

callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot=true)

# Our data

In [15]:
data_dir = "/Users/adrocampos/covid19/synth_data/"
regions = ["2", "3", "5", "10", "15", "20", "30"][1]
mobility_type  = ["inv_dist", "border", "neighbor"][2]
initially_recovered = false

false

In [16]:
positions = CSV.File(data_dir * "positions_" * regions * "_regions.csv")
positions = DataFrame(positions)

Row,id,N,density,x,y
,Int64,Int64,Float64,Float64,Float64
1,1,2780,308.973,0.246368,0.714476
2,2,1081,154.375,0.315868,3.86025


In [17]:
file = "1"

"1"

In [18]:
csv_reader = CSV.File(data_dir * "SIR_" * regions * "_regions_" * mobility_type * "_" * file * ".csv")
df = DataFrame(csv_reader)

Row,t,S1,I1,R1,S2,I2,R2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2773.0,7.0,0.0,1078.0,3.0,0.0
2,0.1,2772.93,6.99984,0.0699992,1077.97,2.99992,0.0299996
3,0.2,2772.86,6.9997,0.139997,1077.94,2.99985,0.0599985
4,0.3,2772.79,6.99959,0.209993,1077.91,2.99979,0.0899967
5,0.4,2772.72,6.99951,0.279989,1077.88,2.99974,0.119994
6,0.5,2772.65,6.99945,0.349984,1077.85,2.9997,0.149992
7,0.6,2772.58,6.99942,0.419978,1077.82,2.99967,0.179988
8,0.7,2772.51,6.99942,0.489972,1077.79,2.99964,0.209985
9,0.8,2772.44,6.99945,0.559966,1077.76,2.99963,0.239981


In [19]:
index = range(1,stop=5001,step=50)
df = df[index,:]

Row,t,S1,I1,R1,S2,I2,R2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2773.0,7.0,0.0,1078.0,3.0,0.0
2,5.0,2769.47,7.02461,3.50337,1076.49,3.00736,1.50088
3,10.0,2765.85,7.11594,7.03573,1074.95,3.03776,3.0112
4,15.0,2762.1,7.27414,10.6305,1073.37,3.09129,4.5425
5,20.0,2758.18,7.49965,14.3211,1071.73,3.16823,6.1064
6,25.0,2754.07,7.79326,18.1415,1070.02,3.26898,7.71469
7,30.0,2749.72,8.15609,22.1259,1068.23,3.39412,9.37944
8,35.0,2745.1,8.58952,26.3093,1066.34,3.54436,11.113
9,40.0,2740.18,9.09517,30.7274,1064.35,3.72052,12.9281


In [20]:
X = Matrix(df[:, [:S1, :I1, :R1]])'
t = df.t

101-element Vector{Float64}:
   0.0
   5.0
  10.0
  15.0
  20.0
  25.0
  30.0
  35.0
  40.0
  45.0
  50.0
  55.0
  60.0
   ⋮
 445.0
 450.0
 455.0
 460.0
 465.0
 470.0
 475.0
 480.0
 485.0
 490.0
 495.0
 500.0

In [21]:
tspan=(t[begin], t[end])
tsteps = range(tspan[1], tspan[2], length = size(t)[1])

u0 = X[:,1]

3-element Vector{Float64}:
 2773.0
    7.0
    0.0

### Neural ODE for our data

In [8]:
rng = Random.default_rng()

dudt2 = Lux.Chain(Lux.Dense(3, 50, tanh),
                  Lux.Dense(50, 50, tanh),
                  Lux.Dense(50, 3))

p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

NeuralODE()         # 2_903 parameters

In [ ]:
function predict_neuralode(p)
  Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, X .- pred)
    return loss, pred
end

# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
        
        
#     plt = scatter(tsteps, X[1,:], label = "data")
#     scatter!(plt, tsteps, pred[1,:], label = "prediction")
        
    pS = scatter(tsteps, X[1,:], label = "data S")
    scatter!(pS, tsteps, pred[1,:], label = "prediction S")
        
    pI = scatter(tsteps, X[2,:], label = "data I")
    scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
    pR = scatter(tsteps, X[3,:], label = "data R")
    scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
    display(plot(pS, pI, pR, layout = (3,1)))
            
        
  end
  return false
end

In [ ]:
pinit = Lux.ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...; doplot=true)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(optprob,
                                       ADAM(0.05),
                                       callback = callback,
                                       maxiters = 300)

In [ ]:




optprob2 = remake(optprob, u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2,
                                        Optim.BFGS(initial_stepnorm=0.01),
                                        callback=callback,
                                        allow_f_increases = false)

callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot=true)

## Universal Differential Equations

In [22]:
ann = Lux.Chain(Lux.Dense(3, 50, tanh),
                  Lux.Dense(50, 50, tanh),
                  Lux.Dense(50, 3))

p, st = Lux.setup(rng, dudt2)

((layer_1 = (weight = Float32[-0.1411977 0.1984585 0.1533245; -0.27679032 0.14865057 0.0018050502; … ; 0.31854424 0.14625117 0.12700589; 0.23811239 0.3146623 -0.32001558], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = Float32[0.07549488 -0.14571092 … -0.061022766 0.17946678; 0.15695886 -0.18745653 … -0.035811204 -0.020084623; … ; -0.0059291576 -0.086555764 … 0.06292852 -0.05709093; -0.049365435 0.06756338 … 0.18552321 0.17794092], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = Float32[0.24020226 -0.18964177 … -0.066434495 -0.22679028; 0.27818596 0.19678861 … 0.14652409 -0.015181786; -0.1775842 -0.23730202 … 0.02610807 -0.055216853], bias = Float32[0.0; 0.0; 0.0;;])), (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()))

In [23]:
# ann = FastChain(FastDense(3, 50, tanh),
#                 FastDense(50, 50, tanh),
#                 FastDense(50, 1))


# # Get the initial parameters, first two is linear birth / decay of prey and predator

# # ps, st = Lux.setup(rng, ann)

# ## Firs the parameters for Beta, gama und N, then the weigths. 
# p = [.1; .1; 2000; initial_params(ann)]

In [24]:
function dudt_(du, u, p, t)
    
    S, I, R = u
    β, γ, N = p[1:3]

    z  = ann(u, p[3:end])
    dS = -β * S * I/N - z[1]  # susceptible
    dI =  β * S * I/N - γ*I - z[1] # infected
    dR =  γ * I

    du[1] = dS
    du[2] = dI
    du[3] = dR

end



# Define the problem
prob_UODE = ODEProblem(dudt_, X, tspan, p) ##prob_neuralode
# ode_data = Array(solve(prob_UODE, Tsit5()))

ODEProblem with uType Adjoint{Float64, Matrix{Float64}} and tType Float64. In-place: true
timespan: (0.0, 500.0)
u0: 3×101 adjoint(::Matrix{Float64}) with eltype Float64:
 2773.0  2769.47     2765.85     …   772.238    758.053   744.13
    7.0     7.02461     7.11594       29.7332    29.186    28.6498
    0.0     3.50337     7.03573     1978.03    1992.76   2007.22

In [25]:
function predict(θ, X=X[:,1], T=t)
    Array(solve(prob_UODE, Vern7(), u0=X, p=θ, tspan=tspan, saveat=T))
    end

predict (generic function with 3 methods)

In [26]:
# function loss(data, pred)
# 	return sum(abs2, data - pred)
# end


function loss(θ)
    X̂ = predict(θ)
    sum(abs2, X - X̂) 
end

loss (generic function with 1 method)

In [ ]:
# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ,args...) = begin
	l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses)%5==0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end


In [ ]:
res1 = DiffEqFlux.sciml_train(loss, p, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10)
println("Training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
p_trained = res1.minimizer

In [ ]:
## Analysis of the trained network
# Interpolate the solution
tsample = t[1]:t[end]
pred = predict(p_trained, X[:,1], tsample)

In [ ]:
pS = scatter(tsteps, X[1,:], label = "data S")
scatter!(pS, tsteps, pred[1,:], label = "prediction S")

pI = scatter(tsteps, X[2,:], label = "data I")
scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
pR = scatter(tsteps, X[3,:], label = "data R")
scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
display(plot(pS, pI, pR, layout = (3,1)))
            

In [ ]:
# res1_uode = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 500)

In [ ]:


# # Do not plot by default for the documentation
# # Users should change doplot=true to see the plots callbacks
# callback = function (p, l, pred; doplot = true)
#   println(l)
#   # plot current prediction against data
#   if doplot
        
        
# #     plt = scatter(tsteps, X[1,:], label = "data")
# #     scatter!(plt, tsteps, pred[1,:], label = "prediction")
        
#     pS = scatter(tsteps, X[1,:], label = "data S")
#     scatter!(pS, tsteps, pred[1,:], label = "prediction S")
        
#     pI = scatter(tsteps, X[2,:], label = "data I")
#     scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
#     pR = scatter(tsteps, X[3,:], label = "data R")
#     scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
#     display(plot(pS, pI, pR, layout = (3,1)))
            
        
#   end
#   return false
# end

In [ ]:
# pinit = Lux.ComponentArray(p)
# callback(pinit, loss_UODE(pinit)...; doplot=true)

# # use Optimization.jl to solve the problem
# adtype = Optimization.AutoZygote()

# optf = Optimization.OptimizationFunction((x, p) -> loss_UODE(x), adtype)
# optprob = Optimization.OptimizationProblem(optf, pinit)

# result_neuralode = Optimization.solve(optprob,
#                                        ADAM(0.05),
#                                        callback = callback,
#                                        maxiters = 300)

In [ ]:
# # The model weights are destructured into a vector of parameters
# p_model = initial_params(model_univ)
# n_weights = length(p_model)

# print(p_model)
# print(n_weights)

# # Parameters of the second equation (linear dynamics)
# p_system = Float32[0.5, -0.5]

# p_all = [p_model; p_system]
# θ = Float32[u0; p_all]

# function dudt_univ!(du, u, p, t)
#     # Destructure the parameters
#     model_weights = p[1:n_weights]
#     α = p[end - 1]
#     β = p[end]

#     # The neural network outputs a control taken by the system
#     # The system then produces an output
#     model_control, system_output = u

#     # Dynamics of the control and system
#     dmodel_control = model_univ(u, model_weights)[1]
#     dsystem_output = α*system_output + β*model_control

#     # Update in place
#     du[1] = dmodel_control
#     du[2] = dsystem_output
# end

# prob_univ = ODEProblem(dudt_univ!, [0f0, u0], tspan, p_all)
# sol_univ = solve(prob_univ, Tsit5(),abstol = 1e-8, reltol = 1e-6)

# function predict_univ(θ)
#   return Array(solve(prob_univ, Tsit5(), u0=[0f0, θ[1]], p=θ[2:end],
#                               saveat = tsteps))
# end

# loss_univ(θ) = sum(abs2, predict_univ(θ)[2,:] .- 1)
# l = loss_univ(θ)